In [20]:
import tokenizers
import pandas as pd
import transformers
from sklearn.model_selection import train_test_split
from datasets import load_dataset, load_from_disk, Dataset
from transformers import (AutoTokenizer, LineByLineTextDataset, BertConfig, BertForMaskedLM, DataCollatorWithPadding, Trainer, TrainingArguments, BertForSequenceClassification,
                        AutoConfig, BertModel, AutoModel, pipeline, AutoModelForSequenceClassification)

# Carregando modelo

In [2]:
#modelo = AutoModel.from_pretrained('seguranca\modelo_data_2022-12-10_hora_00-12')

Inicializando tokenizer:

In [3]:
vocab_size = 1360 #sao 1360 caracteres cuneiformes em unicode no total
max_len = 128
with open('../CLIdata/single-char/single-chars.txt',encoding='utf8') as f:
    single_chars = f.read().split('\n')

original_bert_tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

#treinamento do tokenizer com o arquivo de caracteres
my_tokenizer = original_bert_tokenizer.train_new_from_iterator(single_chars,vocab_size = vocab_size)

Tokenizando frase modelo

In [4]:
my_tokenizer.tokenize('𒀳 𒊹 𒉺 𒊹 𒊕 𒁺 𒀭 𒍣 𒄿 𒉺 𒀭 𒍣 𒄿 𒌋 𒌀 𒀭 𒍣 𒄿 𒋁')

['𒀳',
 '𒊹',
 '𒉺',
 '𒊹',
 '𒊕',
 '𒁺',
 '𒀭',
 '𒍣',
 '𒄿',
 '𒉺',
 '𒀭',
 '𒍣',
 '𒄿',
 '𒌋',
 '𒌀',
 '𒀭',
 '𒍣',
 '𒄿',
 '𒋁']

In [11]:
classifier = pipeline('fill-mask',model='../checkpoints/meu_output_dia_2022-27-11_hora_01-27/checkpoint-10000',tokenizer=my_tokenizer)

In [7]:
classifier('𒋦 𒀀 𒊮 𒊭 𒊹 𒆏 [MASK]')


[{'score': 0.049207624047994614,
  'token': 5,
  'token_str': '𒀀',
  'sequence': '𒋦 𒀀 𒊮 𒊭 𒊹 𒆏 𒀀'},
 {'score': 0.03924712911248207,
  'token': 404,
  'token_str': '𒆏',
  'sequence': '𒋦 𒀀 𒊮 𒊭 𒊹 𒆏 𒆏'},
 {'score': 0.032658085227012634,
  'token': 31,
  'token_str': '𒀚',
  'sequence': '𒋦 𒀀 𒊮 𒊭 𒊹 𒆏 𒀚'},
 {'score': 0.027577795088291168,
  'token': 460,
  'token_str': '𒇇',
  'sequence': '𒋦 𒀀 𒊮 𒊭 𒊹 𒆏 𒇇'},
 {'score': 0.025667162612080574,
  'token': 336,
  'token_str': '𒅋',
  'sequence': '𒋦 𒀀 𒊮 𒊭 𒊹 𒆏 𒅋'}]

In [9]:
simb = '𒋁'
classifier('𒀳 𒊹 𒉺 𒊹 𒊕 𒁺 𒀭 𒍣 𒄿 𒉺 [MASK] 𒍣 𒄿 𒌋 𒌀 𒀭 𒍣 𒄿 𒋁')

[{'score': 0.04920763522386551,
  'token': 5,
  'token_str': '𒀀',
  'sequence': '𒀳 𒊹 𒉺 𒊹 𒊕 𒁺 𒀭 𒍣 𒄿 𒉺 𒀀 𒍣 𒄿 𒌋 𒌀 𒀭 𒍣 𒄿 𒋁'},
 {'score': 0.03924713656306267,
  'token': 404,
  'token_str': '𒆏',
  'sequence': '𒀳 𒊹 𒉺 𒊹 𒊕 𒁺 𒀭 𒍣 𒄿 𒉺 𒆏 𒍣 𒄿 𒌋 𒌀 𒀭 𒍣 𒄿 𒋁'},
 {'score': 0.032658085227012634,
  'token': 31,
  'token_str': '𒀚',
  'sequence': '𒀳 𒊹 𒉺 𒊹 𒊕 𒁺 𒀭 𒍣 𒄿 𒉺 𒀚 𒍣 𒄿 𒌋 𒌀 𒀭 𒍣 𒄿 𒋁'},
 {'score': 0.02757779322564602,
  'token': 460,
  'token_str': '𒇇',
  'sequence': '𒀳 𒊹 𒉺 𒊹 𒊕 𒁺 𒀭 𒍣 𒄿 𒉺 𒇇 𒍣 𒄿 𒌋 𒌀 𒀭 𒍣 𒄿 𒋁'},
 {'score': 0.025667166337370872,
  'token': 336,
  'token_str': '𒅋',
  'sequence': '𒀳 𒊹 𒉺 𒊹 𒊕 𒁺 𒀭 𒍣 𒄿 𒉺 𒅋 𒍣 𒄿 𒌋 𒌀 𒀭 𒍣 𒄿 𒋁'}]

### Criar, salvar e carregar dataset

In [29]:
#novo_dataset.save_to_disk("../CLIdata/datasets/cuneiform-train-val-test.hf")

In [81]:
load_dataset('csv',data_files={'train':'../CLIdata/train-val-test/train.csv','val':'../CLIdata/train-val-test/val.csv','text':'../CLIdata/train-val-test/test.csv'})#novo_dataset = 

Using custom data configuration default-8b999ba03697e609
Found cached dataset csv (C:/Users/igorr/.cache/huggingface/datasets/csv/default-8b999ba03697e609/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 82017
    })
    val: Dataset({
        features: ['text', 'label'],
        num_rows: 9113
    })
    text: Dataset({
        features: ['text', 'label'],
        num_rows: 6895
    })
})

In [15]:
xxx = load_from_disk('../CLIdata/datasets/cuneiform-spaced')

In [16]:
xxx

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 82017
    })
    val: Dataset({
        features: ['text', 'label'],
        num_rows: 9113
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 6895
    })
})

In [79]:
xxx['train']['text']

['𒀸𒋢𒆠',
 '𒋳𒈠𒈗𒁁𒇷𒉿𒄴𒈾𒉡𒉌𒅇',
 '𒊹𒅆𒆕𒊹𒉆𒊹𒅍𒊹𒅆𒆕𒊹𒉆𒊹𒅍',
 '𒋗𒈫𒊹𒌀𒈨𒌍𒃻𒅆𒈨𒌍𒁹𒋻𒋛𒀀𒀀𒇉𒀀𒃻𒄯𒊏',
 '𒇻𒂍𒀸𒁁𒄵𒄴𒋠𒈛𒈠𒌀𒌒𒉌𒌀',
 '𒊹𒃻𒂍𒈬𒈨𒌍𒊹𒌑𒍑𒆠𒌑𒅎𒍇𒇻𒁕𒂍𒌉𒃻',
 '𒂟𒈨𒌍𒀉𒆗𒈨𒌍𒋗𒉏𒋾',
 '𒄑𒆰𒅆𒋫',
 '𒁯𒄯𒊕𒂷𒄷',
 '𒆠𒉺𒌓𒄿𒈾𒇷𒁍𒆠𒉺𒁴𒂊𒁉𒊭𒄠',
 '𒉌𒆗𒃻𒆍𒊹𒀭𒌓𒀭𒅎𒌋𒀭',
 '𒊹𒆳𒀸𒃲𒊹𒈨𒌍',
 '𒀀𒈾𒀭𒌨𒈨𒌈𒋩𒁍𒁀𒉌𒈠',
 '𒊹𒌗𒊭𒌗𒉈𒌗𒆥𒀸𒄿𒂵𒊑',
 '𒀝𒃲𒌓𒈠𒀊𒋫𒊒𒌓',
 '𒆠𒄑𒄩𒇻𒂠𒁕𒊑𒋫𒌨𒊕𒃲',
 '𒀸𒊕𒁺𒀭𒅅𒋢𒁄𒋢',
 '𒅇𒈗𒈨𒌍𒆳𒉡𒄩𒀸𒊺',
 '𒀀𒈾𒀀𒊮𒃵𒑚𒍑𒈭𒊹',
 '𒀭𒅎𒂙𒊏𒄑',
 '𒊹𒂁𒆪𒌓𒁉𒈨𒌍',
 '𒇥𒅗𒂵',
 '𒃲𒋾𒍪𒂊𒌋𒅗𒂗𒃲𒀭𒄭𒂵𒄿',
 '𒊹𒌫𒊹𒍠𒋫𒊮𒀬𒈨𒌍',
 '𒄑𒊓𒄭𒅖𒈠𒇷𒇻𒌋𒉌',
 '𒀭𒋻𒀭𒋻𒄖𒌓𒈠𒀸',
 '𒉡𒍝𒁁𒀸𒊮𒌷',
 '𒋗𒃮𒌆',
 '𒅇𒋳𒈠𒊹𒄞',
 '𒄣𒆷𒉏𒈠',
 '𒊭𒌷𒌒𒉻𒁕𒀀𒈾𒀀𒄷𒆷𒀀𒈾𒀀𒊑',
 '𒍏𒆠𒈗𒁺',
 '𒁹𒀭𒂗𒆤𒀉𒆗𒉡𒀀𒃻𒁹𒀭𒆗𒆗𒈬𒁹𒆐𒋙𒀭𒂗𒆤𒀀𒃻𒁹𒄿𒀭𒂗𒆤',
 '𒀀𒈾𒌋𒌋𒂗𒐊',
 '𒆥𒈾𒌈𒅗𒌤𒊭𒋢𒌑𒄖𒁺𒊹𒋗𒄴𒄷𒁕𒀜',
 '𒄭𒁉𒊹𒋫𒈥𒊹𒋗𒌓𒆟𒊹𒋫𒈥𒊹𒅗𒂵𒊏𒌈',
 '𒊹𒃷𒑛𒊬𒊹',
 '𒀀𒈾𒆪𒀀𒈾𒌨𒁲𒅀𒁕𒀊𒌅𒇷𒁍𒍑',
 '𒁹𒌋𒌋𒉡𒆠𒄿𒀭𒀸𒁁𒇽𒈬𒁺𒌉𒃻𒀀',
 '𒅗𒅗𒆷𒀾𒋗',
 '𒋢𒈠𒀀𒄩',
 '𒂗𒉌𒌨𒊒𒁍',
 '𒅇𒀭𒉡𒍑𒄿𒈾𒀭𒈾𒀭𒋾𒆠',
 '𒉌𒌓𒈩𒊹𒈾𒄭𒊏𒇽𒈬𒆸𒋢𒉺𒈨𒌍',
 '𒆠𒄿𒊭𒈠𒆷𒊹𒋙',
 '𒅎𒍇𒇻𒅎𒋛𒁲𒅎𒆳𒊏𒅎𒈥𒌅',
 '𒆠𒋛𒍝𒀝',
 '𒀭𒂔𒌑𒀀𒀀𒈾𒀭𒅟𒈬𒅟𒇸𒇷',
 '𒀭𒀫𒌓𒀸𒂍𒊕𒅍𒀸𒊕𒈬𒀸𒌓𒁺𒋙𒅆𒈨𒌍𒋙𒌓𒉆𒌀𒈨𒌍𒌦𒈨𒌍𒌉𒈨𒌍',
 '𒌓𒊹𒈬𒊹𒁹𒁕𒊑𒅀𒀀𒈲𒈗𒆳𒆳',
 '𒀀𒊮𒂊𒊒𒋙𒆷𒉌𒈬𒇻𒊺𒆰𒌒𒁍𒇻𒆷𒄿𒈾𒅆𒌋',
 '𒆷𒉌𒌓𒂊𒉡𒈠𒆕𒋙𒁉𒇽𒁉𒁷𒈾𒆷𒄿𒅆',
 '𒀭𒈗𒌅𒃮𒇺',
 '𒊒𒌑𒄑𒍪',
 '𒊹𒆠𒊕𒁍𒄩',
 '𒊭𒀠𒋗𒉡',
 '𒈥𒄿𒁀𒋙𒉌',
 '𒈲𒁻',
 '𒇻𒀀𒁔',
 '𒄿𒈾𒍣𒊑𒅇',
 '𒁁𒂖𒋾𒅀',
 '𒁉𒅎',
 '𒀭𒊩𒌆𒆸𒁀',
 '𒄿𒉺𒅁𒊏𒈠𒀀𒋧𒈠𒇽𒈜𒈨𒌍',
 '𒊭𒊹𒆰𒆕𒇽𒊹𒂍𒃲',
 '𒀀𒋻𒅗𒅗',
 '𒁹𒅀𒋢𒊑𒊹',
 '𒆗𒆳𒅀𒆠𒄿𒄩𒉈𒅅𒁍𒉌',
 '𒅆𒊹𒆰𒊹𒇽𒃲𒍑𒁇𒈨',
 '𒃷𒈨',
 '𒀠𒇷𒃶𒉌𒀀𒁕𒃲',
 '𒄑𒈬𒌈𒁕'

# Extra (testes)

In [ ]:
'''
sentences = ['𒋦𒀀𒊮𒊭𒊹𒆏𒅀','𒀳𒊹𒉺𒊹𒊕𒁺𒀭𒍣𒄿𒉺𒀭𒍣𒄿𒌋𒌀𒀭𒍣𒄿𒋁']
token_obj = my_tokenizer(sentences, truncation=True)
tokenization = my_tokenizer.tokenize(sentences)
print(token_obj)
print(50*'-','Sentence 1',50*'-')
print("Original sentence:",sentences[0])
print("Tokenization:", my_tokenizer.tokenize(sentences[0]))
print("Encoding:", token_obj.input_ids[0])
print('Decoding:',my_tokenizer.decode(token_obj.input_ids[0]))
print(50*'-','Sentence 2',50*'-')
print("Original sentence:",sentences[1])
print("Tokenization:", my_tokenizer.tokenize(sentences[1]))
print("Encoding:", token_obj.input_ids[1])
print('Decoding:',my_tokenizer.decode(token_obj.input_ids[1]))
'''

{'input_ids': [[2, 6276, 734, 625, 3], [2, 6297, 667, 4097, 5088, 1261, 1840, 581, 787, 5088, 782, 3]], 'token_type_ids': [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}
-------------------------------------------------- Sentence 1 --------------------------------------------------
Original sentence: 𒋦𒀀𒊮𒊭𒊹𒆏𒅀
Tokenization: ['𒋦𒀀𒊮𒊭𒊹', '##𒆏', '##𒅀']
Encoding: [2, 6276, 734, 625, 3]
Decoding: [CLS] 𒋦𒀀𒊮𒊭𒊹𒆏𒅀 [SEP]
-------------------------------------------------- Sentence 2 --------------------------------------------------
Original sentence: 𒀳𒊹𒉺𒊹𒊕𒁺𒀭𒍣𒄿𒉺𒀭𒍣𒄿𒌋𒌀𒀭𒍣𒄿𒋁
Tokenization: ['𒀳𒊹', '##𒉺', '##𒊹𒊕𒁺', '##𒀭𒍣𒄿', '##𒉺𒀭', '##𒍣𒄿', '##𒌋', '##𒌀', '##𒀭𒍣𒄿', '##𒋁']
Encoding: [2, 6297, 667, 4097, 5088, 1261, 1840, 581, 787, 5088, 782, 3]
Decoding: [CLS] 𒀳𒊹𒉺𒊹𒊕𒁺𒀭𒍣𒄿𒉺𒀭𒍣𒄿𒌋𒌀𒀭𒍣𒄿𒋁 [SEP]


In [ ]:
#fill_mask = transformers.pipeline(
#    "fill-mask",
#    model=test,
#    tokenizer=my_tokenizer
#)

# Novos testes

In [18]:
dataset = load_from_disk('../CLIdata/datasets/cuneiform-spaced/')

In [19]:
my_tokenizer = original_bert_tokenizer.train_new_from_iterator(dataset['train']['text'] + dataset['val']['text'] + dataset['test']['text'],vocab_size = vocab_size)

In [23]:
model = AutoModelForSequenceClassification.from_pretrained('../checkpoints/meu_output_dia_2022-02-11_hora_02-02/checkpoint-20000')

Some weights of the model checkpoint at ../checkpoints/meu_output_dia_2022-02-11_hora_02-02/checkpoint-20000 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from 